In [1]:
import os
from dotenv import load_dotenv
from slack_sdk import WebClient

load_dotenv()

SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
CHANNEL_NAME = os.getenv("CHANNEL_NAME")

# 슬랙 봇 토큰
SLACK_BOT_TOKEN = SLACK_BOT_TOKEN

In [2]:
import os
import time
import re
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

# Slack 클라이언트 초기화
client = WebClient(token=SLACK_BOT_TOKEN)  # 직접 토큰 입력 (환경 변수 대신)

In [ ]:
def send_message(message: str, channel: str):
    try:
        client.chat_postMessage(channel=channel, text=message)
        # print("메시지가 전송되었습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")


def crawl_with_selenium() -> str:
    """
    Selenium으로 coinness.com/article에서 
    특정 CSS 셀렉터(#root > ... > .ArticleListContainer-sc-cj3rkv-0.fkxjqP)
    영역의 텍스트만 추출.
    """
    driver = webdriver.Chrome()
    driver.get("https://coinness.com/article")
    time.sleep(5)  # JS 로딩 대기
    
    css = "#root > div > div.Wrap-sc-v065lx-0.hwmGSB > div > main > div.ContentContainer-sc-91rcal-0.jJHYjq > div.ArticleListContainer-sc-cj3rkv-0.fkxjqP"
    try:
        container = driver.find_element(By.CSS_SELECTOR, css)
        return container.text
    except Exception as e:
        print(f"[ERROR] 요소 찾기 실패: {e}")
        return ""
    finally:
        driver.quit()

# 파싱싱
def parse_news_text(raw_text: str) -> pd.DataFrame:
    """
    1) 'HH:MM' 형식을 만나면 새 기사 시작
    2) 바로 다음 줄 = 날짜
    3) 그 다음 줄 = 제목
    4) 이후로 같은 패턴(시간)이 나오기 전까지 = 본문
    결과: DataFrame[time, date, title, content]
    """
    lines = [line.strip() for line in raw_text.splitlines() if line.strip()]
    time_pattern = re.compile(r'^\d{2}:\d{2}$')

    articles = []
    current_time = None
    current_date = None
    current_title = None
    content_lines = []

    def save_article():
        if current_time and current_date and current_title:
            articles.append({
                "time": current_time,
                "date": current_date,
                "title": current_title,
                "content": "\n".join(content_lines).strip()
            })

    state = "idle"
    for line in lines:
        # (1) 새로운 시간(기사 시작 신호) 감지
        if time_pattern.match(line):
            save_article()
            current_time = line
            current_date = None
            current_title = None
            content_lines = []
            state = "got_time"
            continue
        
        # (2) got_time → 날짜
        if state == "got_time":
            current_date = line
            state = "got_date"
            continue
        
        # (3) got_date → 제목
        if state == "got_date":
            current_title = line
            state = "got_title"
            continue
        
        # (4) 나머지 줄은 본문
        content_lines.append(line)
        state = "collecting_content"

    # 마지막 기사 저장
    save_article()

    df = pd.DataFrame(articles, columns=["time", "date", "title", "content"])
    return df

def process_and_send(new_df: pd.DataFrame, csv_path: str, channel_name: str):
    """
    1) 기존 CSV를 로드해 (time, date, title, content) 집합 생성
    2) new_df 각 행을 순회해, 기존에 없으면 '새 기사'로 간주 -> Slack 메시지
    3) 모든 기사(기존 + 새 기사) 합쳐 CSV 최종 저장
    """

    if new_df.empty:
        print("[INFO] 새로 추출된 기사가 없습니다.")
        return
    
    # 1) 기존 CSV 로드 (없으면 빈 DF)
    if os.path.exists(csv_path):
        existing_df = pd.read_csv(csv_path)
    else:
        existing_df = pd.DataFrame(columns=["time", "date", "title", "content"])
    
    # 집합 생성
    existing_signatures = set()
    for _, row in existing_df.iterrows():
        sig = (row["time"], row["date"], row["title"], row["content"])
        existing_signatures.add(sig)
    
    # 2) 새 기사 중 '기존에 없는' 것만 골라 Slack 전송
    new_articles_list = []
    
    for _, row in new_df.iterrows():
        sig = (row["time"], row["date"], row["title"], row["content"])
        if sig not in existing_signatures:
            # 완전히 새 기사
            new_articles_list.append(row)
            existing_signatures.add(sig)  # 앞으로 중복 방지
    
    if not new_articles_list:
        print("[INFO] 추가된 새 기사가 없습니다.")
    else:
        # Slack 메시지
        message_parts = []
        for row in new_articles_list:
            part = (
                f"[{row['time']}][{row['date']}]\n"
                f"{row['title']}\n"
                f"{row['content']}\n"
            )
            message_parts.append(part)
        
        final_msg = "".join(message_parts)
        
        # Slack 전송
        send_message(final_msg, channel_name)
        
        print(f"[INFO] 새 기사 {len(new_articles_list)}건이 Slack에 전송되었습니다.")

    # 3) 기존 DF + 새 기사 DF -> 최종 저장
    # 새 기사 리스트를 DF로 변환
    new_articles_df = pd.DataFrame(new_articles_list, columns=["time", "date", "title", "content"])
    
    # 기존 DF + 새 기사 DF 병합
    final_df = pd.concat([existing_df, new_articles_df], ignore_index=True)
    # 중복 제거 (혹시 모를 중복)
    final_df.drop_duplicates(subset=["time", "date", "title", "content"], inplace=True)
    
    final_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"[INFO] CSV 저장 완료 (총 {len(final_df)}건).")

def run_autoupdate(interval: int = 1800, channel_name: str = CHANNEL_NAME):
    news_folder = "NEWS"
    os.makedirs(news_folder, exist_ok=True)
    csv_path = os.path.join(news_folder, "my_news.csv")
    
    print(f"[INFO] 자동 업데이트 시작 (간격: {interval}초).")
    last_date = datetime.date.today()

    try:
        while True:
            now = datetime.datetime.now()
            today_date = now.date()
            
            # (A) 날짜가 바뀌었으면 CSV 초기화
            if today_date != last_date:
                if os.path.exists(csv_path):
                    os.remove(csv_path)
                    print(f"[INFO] 날짜 변경으로 CSV({csv_path})를 초기화했습니다.")
                last_date = today_date

            print(f"\n[INFO] 뉴스 확인 중... ({now.strftime('%Y-%m-%d %H:%M:%S')})")
            
            # (B) 크롤링 -> 파싱
            raw_text = crawl_with_selenium()
            df_new = parse_news_text(raw_text)
            
            # (C) 새 기사만 Slack 전송 후, CSV 병합 저장
            process_and_send(df_new, csv_path, channel_name)

            print(f"[INFO] {interval}초 뒤 다음 업데이트 진행...")
            time.sleep(interval)

    except KeyboardInterrupt:
        print("[INFO] 자동 업데이트 종료 (KeyboardInterrupt).")

#################################
# 메인 실행
#################################
if __name__ == "__main__":
    run_autoupdate(interval=300, channel_name=CHANNEL_NAME)


[INFO] 자동 업데이트 시작 (간격: 300초).

[INFO] 뉴스 확인 중... (2025-01-23 01:16:44)
[INFO] 추가된 새 기사가 없습니다.
[INFO] CSV 저장 완료 (총 10건).
[INFO] 300초 뒤 다음 업데이트 진행...

[INFO] 뉴스 확인 중... (2025-01-23 01:21:59)
[INFO] 새 기사 2건이 Slack에 전송되었습니다.
[INFO] CSV 저장 완료 (총 12건).
[INFO] 300초 뒤 다음 업데이트 진행...

[INFO] 뉴스 확인 중... (2025-01-23 01:27:10)
[INFO] 추가된 새 기사가 없습니다.
[INFO] CSV 저장 완료 (총 12건).
[INFO] 300초 뒤 다음 업데이트 진행...

[INFO] 뉴스 확인 중... (2025-01-23 01:32:20)
[INFO] 추가된 새 기사가 없습니다.
[INFO] CSV 저장 완료 (총 12건).
[INFO] 300초 뒤 다음 업데이트 진행...

[INFO] 뉴스 확인 중... (2025-01-23 01:37:30)
[INFO] 추가된 새 기사가 없습니다.
[INFO] CSV 저장 완료 (총 12건).
[INFO] 300초 뒤 다음 업데이트 진행...
